In [ ]:
import os
import zipfile

with zipfile.ZipFile('/content/sketchdata.zip', 'r') as ref:
  ref.extractall('/content/sketchdata')

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms

In [ ]:
image_dir = '/content/sketchdata/photo'
sketch_dir = '/content/sketchdata/original_sketch'
x = np.load(dir)

IsADirectoryError: ignored

In [ ]:
# load images in
class images(Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.X)
  
  def __getitem__(self, index):
    image = self.X[index]
    X = self.transform(image)
    return X

In [ ]:
transform = transforms.Compose([transforms.RandomResizedCrop(size = (224, 224)),
                                transforms.ToTensor()])

In [ ]:
class Generator(nn.Module):

  def __init__(self):

    super(Generator, self).__init__()

    self.first_stage = torch.nn.Conv2d(224, 224, 32)

    self.second_stage = torch.nn.Conv2d(28, 28, 256)
    self.third_stage = torch.nn.Conv2d(56, 56, 128)
    self.fourth_stage = torch.nn.Conv2d(112, 112, 64)
    one = torch.nn.Conv2d(1,1)
    three = torch.nn.Conv2d(3,3)
    m = torch.nn.Upsample(scale_factor=2, mode='bilinear')


  def forward(self, img):

    feature_maps = []

    output_img = torch.zeros(())

    for i in range(1, N - 1):
      f_m = feature_maps[i]
      upsampled_fm = m(f_m)
      f_m_1 = feature_maps[i + 1]
      upsampled_fm = torch.cat((f_m, upsampled_fm))
      ones_conv = one(upsampled_fm)
      threes_conv = three(ones_conv)

      feature_maps[i + 1] =  threes_conv


    return feature_maps[-1]




In [ ]:
def adversarial_loss(generator, discriminator, photo_data):
    # generate fake sketches from photo input
    sketch_data = generator(photo_data)

    # compute discriminator outputs for real and fake sketches
    real_outputs = discriminator(sketch_data)
    fake_outputs = torch.ones_like(real_outputs)

    # compute adversarial loss for generator
    adversarial_loss = F.binary_cross_entropy_with_logits(real_outputs, fake_outputs)

    return adversarial_loss

In [ ]:
class Discriminator(nn.Module):

  def __init__(self):

    self.first_layer = torch.nn.Conv2d(3, 32, 3, stride = 2, padding = 1)
    self.batch1 = torch.nn.BatchNorm2d(32, eps=1e-10)
    self.second_layer = torch.nn.Conv2d(32, 32, 3, stride = 2, padding = 1)
    self.batch2 = torch.nn.BatchNorm2d(32, eps=1e-10)
    self.third_layer = torch.nn.Conv2d(32, 64, 3, stride = 2, padding = 1)
    self.batch3 = torch.nn.BatchNorm2d(64, eps=1e-10)
    self.fourth_layer = torch.nn.Conv2d(64, 64, 3, stride = 2, padding = 1)
    self.batch4 = torch.nn.BatchNorm2d(64, eps=1e-10)
    self.fifth_layer = torch.nn.Conv2d(64, 128, 3, stride = 2, padding = 1)
    self.batch5 = torch.nn.BatchNorm2d(128, eps=1e-10)
    self.sixth_layer = torch.nn.Conv2d(128, 1, 3, stride = 1, padding = 1)
    self.batch6 = torch.nn.BatchNorm2d(1, eps=1e-10)
    self.sigmoid = torch.nn.sigmoid()
  
  def forward(self, img):

    first = F.relu(self.batch1(self.first_layer(img)))
    second = F.relu(self.batch2(self.second_layer(first)))
    third = F.relu(self.batch3(self.third_layer(second)))
    fourth = F.relu(self.batch4(self.fourth_layer(third)))
    fifth = F.relu(self.batch5(self.fifth_layer(fourth)))
    sixth = F.relu(self.batch6(self.sixth_layer(fifth)))
    output = self.sigmoid(sixth)


    return output


In [ ]:
from google.colab import drive
drive.mount('/content/drive')